# Darknet

In [ ]:
# based on https://colab.research.google.com/drive/1_GdoqCJWXsChrOiY8sZMr_zbr_fH-0Fg?usp=sharing#scrollTo=GQQrAMdXN22a
# clone darknet repo
!git clone https://github.com/AlexeyAB/darknet

In [ ]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile

In [ ]:
# verify CUDA
!/usr/local/cuda/bin/nvcc --version

In [ ]:
# make darknet (builds darknet so that you can then use the darknet executable file to run or train object detectors)
!make

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [110]:
# define helper functions
def imShow(path):
  import cv2
  import matplotlib.pyplot as plt
  %matplotlib inline

  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# use this to upload files
def upload():
  from google.colab import files
  uploaded = files.upload() 
  for name, data in uploaded.items():
    with open(name, 'wb') as f:
      f.write(data)
      print ('saved file', name)

# use this to download a file  
def download(path):
  from google.colab import files
  files.download(path)

In [ ]:
# !unzip /content/drive/My\ Drive/20200722/task_garbage_detection_2-2020_07_23_13_39_54-yolo-1.1.zip -d /content/darknet/data/
!unzip /content/drive/My\ Drive/20200722/task_garbage_det_fps_15_img_annot_322-2020_07_24_13_19_02-yolo-1.1.zip -d /content/darknet/data/

In [9]:
!cp /content/drive/My\ Drive/20200722/yolov4-custom.cfg /content/darknet/cfg/

In [ ]:
!rm /content/backup/*

In [114]:
!./darknet detector train data/obj.data cfg/yolov4-custom.cfg yolov4.conv.137 -dont_show -map

v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 139 Avg (IOU: 0.851786, GIOU: 0.849271), Class: 0.995378, Obj: 0.919470, No Obj: 0.003687, .5R: 1.000000, .75R: 0.942857, count: 35, class_loss = 1.110456, iou_loss = 41.971745, total_loss = 43.082199 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 150 Avg (IOU: 0.850754, GIOU: 0.846098), Class: 0.997330, Obj: 0.945543, No Obj: 0.017881, .5R: 1.000000, .75R: 0.884058, count: 69, class_loss = 0.905769, iou_loss = 22.983297, total_loss = 23.889067 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 161 Avg (IOU: 0.691550, GIOU: 0.671181), Class: 0.991318, Obj: 0.728409, No Obj: 0.007409, .5R: 0.923077, .75R: 0.230769, count: 13, class_loss = 0.581209, iou_loss = 0.581517, total_loss = 1.162726 
 total_bbox = 4538083, rewritten_bbox = 0.005311 % 
v3 (iou loss, Normalizer: (iou: 0.07, cls: 1.00) Region 139 Avg (IOU: 0.869799, GIOU: 0.867182), Class: 0.995186, Obj: 0.953230, No Obj: 0.005151, .5R: 1.000000, .75R: 0.921569

In [ ]:
imShow('chart.png')

In [ ]:
!cp cfg/yolov4-custom.cfg /content/drive/My\ Drive/20200722/yolov4-custom.cfg

In [ ]:
!mkdir /content/drive/My\ Drive/20200722/yolow_backup

In [121]:
!cp /content/darknet/backup/* /content/drive/My\ Drive/20200724/yolo_backup

# Detections

### Import existing model

In [115]:
_data_path = 'data/obj.data'
_cfg_path = 'cfg/yolov4-custom.cfg'
_weight_path = '/content/drive/My\ Drive/20200722/yolow_backup/yolov4-custom_best.weights'

In [ ]:
img_path = '/content/fps_5_frame_0014.jpg'
# !./darknet detector map data/obj.data cfg/yolov4-custom.cfg /content/darknet/backup/yolov4-custom_last.weights {img_path} -thresh 0.3 -dont_show #-out /content/my_img.jpg
# !./darknet detector test data/obj.data cfg/yolov4-custom.cfg \
#         /content/darknet/backup/yolov4-custom_last.weights {img_path} \
#         -thresh 0.1 -dont_show #-out /content/my_img.jpg
!./darknet detector test data/obj.data cfg/yolov4-custom.cfg \
        {weight_path} {img_path} \
        -thresh 0.1 -dont_show #-out /content/my_img.jpg

imShow('predictions.jpg')

In [ ]:
video_path = '/content/drive/My\ Drive/20200724/VID_20200722_115436_stabiilizo_annot.mp4'
!./darknet detector demo data/obj.data cfg/yolov4-custom.cfg \
    /content/darknet/backup/yolov4-custom_last.weights -dont_show {video_path} -i 0 -out_filename /content/results.avi -thresh 0.1

In [119]:
!cp ./results.avi /content/drive/My\ Drive/20200722/

^C


In [ ]:
!cp data/obj.data /content/drive/My\ Drive/20200722/yolow_backup
!cp cfg/yolov4-custom.cfg /content/drive/My\ Drive/20200722/yolow_backup

# Generate annotations

In [16]:
import os
import shutil

## Convert Video to frames and create frame_list.txt 

In [97]:
def create_empty_dir(dir):
    if os.path.isdir(dir):
        shutil.rmtree(dir)    
    os.mkdir(dir)

In [98]:
%%capture
# Video to frames
frame_dir = '/content/frames'
input_video = '/content/VID_20200722_115436_stabiilizo_annot.mp4'
# clear folder if exsists
# if os.path.isdir(frame_dir):
#     shutil.rmtree(frame_dir)    
# os.mkdir(frame_dir)
create_empty_dir(frame_dir)

fps = 15
frames_name = os.path.join(frame_dir, "fps_{}_frame_%04d.jpg".format(fps))
!ffmpeg -i {input_video} -vf fps={fps} -qscale:v 2 {frames_name}

In [102]:
from os import listdir
from os.path import isfile, join

frame_list_file = '/content/frames_list.txt'

frame_list = [join(frame_dir, f) for f in listdir(frame_dir) if isfile(join(frame_dir, f)) and f.split(".")[-1] == 'jpg']
print(len(frame_list))
# print(frame_list)

with open(frame_list_file, 'w') as f:
    for frame_path in frame_list:
        f.write(frame_path + "\n")

322


In [103]:
!./darknet detector test {_data_path} {_cfg_path} {_weight_path} -dont_show -out /content/result.json -ext_output <{frame_list_file}> /content/result.txt -thresh 0.1

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, CUDNN_HALF=1, GPU count: 1  
 OpenCV version: 3.2.0
 0 : compute_capability = 600, cudnn_half = 0, GPU: Tesla P100-PCIE-16GB 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    416 x 416 x   3 ->  416 x 416 x  32 0.299 BF
   1 conv     64       3 x 3/ 2    416 x 416 x  32 ->  208 x 208 x  64 1.595 BF
   2 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   3 route  1 		                           ->  208 x 208 x  64 
   4 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   5 conv     32       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  32 0.177 BF
   6 conv     64       3 x 3/ 1    208 x 208 x  32 ->  208 x 208 x  64 1.595 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 208 x 208 x  64 0.003 BF
   8 conv     64       1 x 1/ 1    208 x 208 x  64 ->  208 x 208 x  64 0.354 BF
   9 route  8 2 	                           ->  208 x 208 x 128

## Convert JSON to Pascal VOC

In [58]:
!pip install pascal-voc-writer

In [104]:
import json
from pprint import pprint
from PIL import Image
from pascal_voc_writer import Writer

In [105]:
with open('/content/result.json') as f:
    data = json.load(f)
# pprint(data)

In [107]:
# clear folder if exsists
annot_dir = '/content/pascal_voc'

# if os.path.isdir(annot_dir):
#     shutil.rmtree(annot_dir)    
# os.mkdir(annot_dir)
create_empty_dir(annot_dir)

for det_result in data:
    # pprint(image_res)
    # writer = Writer('path/to/img.jpg', 800, 400)

    img_path = det_result['filename']
    img = Image.open(img_path)
    width, height = img.size

    # Writer(path, width, height)
    writer = Writer(img_path, width, height)

    for obj in det_result['objects']:
        bb_x_center = obj['relative_coordinates']['center_x']
        bb_y_center = obj['relative_coordinates']['center_y']
        
        bb_width = obj['relative_coordinates']['width']
        bb_height = obj['relative_coordinates']['height']

        xmin = int((bb_x_center - bb_width/2) * width)
        xmax = int((bb_x_center + bb_width/2) * width)
        
        ymin = int((bb_y_center - bb_height/2) * height)
        ymax = int((bb_y_center + bb_height/2) * height)

        # ::addObject(name, xmin, ymin, xmax, ymax)
        writer.addObject(obj['name'], xmin, ymin, xmax, ymax)

    #image name without extension
    xml_name = img_path.split("/")[-1].split('.')[0] + ".xml"
    xml_path = os.path.join(annot_dir, xml_name)
    writer.save(xml_path)
    # print(xml_name)

In [ ]:
!cd /content/; zip frames/pascal_voc.zip pascal_voc/*
!cd /content/; zip garbage_det_fps_15_img_annot_322.zip frames/*

In [109]:
!cp /content/garbage_det_fps_15_img_annot_322.zip /content/drive/My\ Drive/20200722

# Pytorch

In [ ]:
%cd /content/
!git clone --depth 1 https://github.com/Tianxiaomo/pytorch-YOLOv4

In [ ]:
# !pip install -U setuptools
# !apt install enchant
# !pip install pyenchant

In [ ]:
# !pip3 install skimage
# !pip install scikit-image
!pip install tensorboardX

In [ ]:
%cd /content/pytorch-YOLOv4/
!pip install -r requirements.txt

In [ ]:
!python train.py -g 0 -dir /content/pytorch-YOLOv4/data

In [ ]:
%cd /content/pytorch-YOLOv4

In [128]:
_data_path = 'data/obj.data'
_cfg_path = '/content/darknet/cfg/yolov4-custom.cfg'
_weight_path = '/content/drive/My\ Drive/20200722/yolow_backup/yolov4-custom_best.weights'
_img_path = '/content/fps_5_frame_0002.jpg'

In [ ]:
!python demo.py -cfgfile {_cfg_path} -weightfile {_weight_path} -imgfile {_img_path}

In [ ]:
# !cat /content/pytorch-YOLOv4/data/train.txt